In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
steepness = 50    # steepness factor
num_blocks = 1000 # number of expression blocks
num_data = 5000   # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_low, p_high = 1.0, 8.0
a_low, a_high = 0.5, 4.5
p_train = np.random.uniform(p_low, p_high, (train_size, 1)).astype(np.float32)
p_test  = np.random.uniform(p_low, p_high, (test_size, 1)).astype(np.float32)
p_dev   = np.random.uniform(p_low, p_high, (val_size, 1)).astype(np.float32)
a_train = np.random.uniform(a_low, a_high, (train_size, num_blocks)).astype(np.float32)
a_test  = np.random.uniform(a_low, a_high, (test_size, num_blocks)).astype(np.float32)
a_dev   = np.random.uniform(a_low, a_high, (val_size, num_blocks)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msRosenbrock
model = msRosenbrock(steepness, num_blocks, timelimit=60)

## Heuristic - N1

In [7]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


In [8]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # set params
    model_heur.set_param_val({"p":p, "a":a})
    # solve
    tick = time.time()
    xval, objval = model_heur.solve("scip")
    tock = time.time()
    # eval
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model_heur.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_heur_n1_50-1000.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:59:36<00:00,  7.18s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+03            1000.0   1000.000000
mean   1.249891e+14               0.0      7.164278
std    2.165959e+14               0.0      1.811283
min    6.677052e+03               0.0      5.159213
25%    8.147731e+03               0.0      5.879934
50%    9.319791e+03               0.0      6.702751
75%    1.249802e+14               0.0      7.822459
max    4.999892e+14               0.0     20.008771
Number of infeasible solution: 0


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 10000 # weight of constraint violation penealty
hlayers_sol = 4        # number of hidden layers for solution mapping
hlayers_rnd = 4        # number of hidden layers for solution mapping
hsize = 2048           # width of hidden layers for solution mapping
lr = 1e-3              # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 143429.63
Epoch 1, Validation Loss: 115900.00
Epoch 2, Validation Loss: 127993.11
Epoch 3, Validation Loss: 63064.74
Epoch 4, Validation Loss: 644523.97
Epoch 5, Validation Loss: 938724.74
Epoch 6, Validation Loss: 83263.77
Epoch 7, Validation Loss: 4122373.37
Epoch 8, Validation Loss: 46205.25
Epoch 9, Validation Loss: 414026.08
Epoch 10, Validation Loss: 37387.88
Epoch 11, Validation Loss: 19799.74
Epoch 12, Validation Loss: 47740.83
Epoch 13, Validation Loss: 16443.97
Epoch 14, Validation Loss: 58899.39
Epoch 15, Validation Loss: 11640.24
Epoch 16, Validation Loss: 13527.90
Epoch 17, Validation Loss: 15559.42
Epoch 18, Validation Loss: 15985.22
Epoch 19, Validation Loss: 10621.04
Epoch 20, Validation Loss: 10488.57
Epoch 21, Validation Loss: 12285.20
Epoch 22, Validation Loss: 11600.55
Epoch 23, Validation Loss: 10271.51
Epoch 24, Validation Loss: 10304.57
Epoch 25, Validation Loss: 9358.16
Epoch 26, Validation Loss: 8967.97
Epoch 27, Validation Loss: 10550

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lr_50-1000.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:20<00:00,  7.10it/s]


            Obj Val  Constraints Viol  Elapsed Time
count   1000.000000       1000.000000   1000.000000
mean    6357.728138          5.738670      0.019345
std     2310.145717         32.967547      0.002545
min     2902.516872          0.000000      0.002996
25%     4949.477974          0.000000      0.019212
50%     5894.668679          0.000000      0.019745
75%     6872.249410          0.000000      0.020049
max    16660.015531        492.800426      0.022525
Number of infeasible solution: 49


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 1000 # weight of constraint violation penealty
hlayers_sol = 4       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 2048          # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmRosenbrock
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_blocks+1, outsize=2*num_blocks, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["p", "a"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=3*num_blocks+1, hidden_dims=[hsize]*hlayers_rnd, output_dim=2*num_blocks)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x"],  output_keys=["x_rnd"], 
                          int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmRosenbrock(["p", "a", "x_rnd"], steepness, num_blocks)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 119191.00
Epoch 1, Validation Loss: 103242.50
Epoch 2, Validation Loss: 49453.37
Epoch 3, Validation Loss: 39898.60
Epoch 4, Validation Loss: 35072.94
Epoch 5, Validation Loss: 21384.19
Epoch 6, Validation Loss: 38918.05
Epoch 7, Validation Loss: 30468.62
Epoch 8, Validation Loss: 25291.34
Epoch 9, Validation Loss: 22412.84
Epoch 10, Validation Loss: 33291.81
Epoch 11, Validation Loss: 15737.15
Epoch 12, Validation Loss: 15881.57
Epoch 13, Validation Loss: 16797.99
Epoch 14, Validation Loss: 20727.59
Epoch 15, Validation Loss: 14368.88
Epoch 16, Validation Loss: 13701.97
Epoch 17, Validation Loss: 21516.69
Epoch 18, Validation Loss: 12356.11
Epoch 19, Validation Loss: 13896.57
Epoch 20, Validation Loss: 19742.80
Epoch 21, Validation Loss: 12957.02
Epoch 22, Validation Loss: 66736.64
Epoch 23, Validation Loss: 29196.55
Epoch 24, Validation Loss: 9596.76
Epoch 25, Validation Loss: 2896213.92
Epoch 26, Validation Loss: 8998.97
Epoch 27, Validation Loss: 9084.63
E

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for p, a in tqdm(list(zip(p_dev, a_dev))):
    # data point as tensor
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32).to("cuda"), 
                  "a": torch.tensor(np.array([a]), dtype=torch.float32).to("cuda"),
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"p":p, "a":a})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(2*num_blocks):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(p)+list(a))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/rb_lt_50-1000.csv")

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:22<00:00,  7.03it/s]


            Obj Val  Constraints Viol  Elapsed Time
count   1000.000000       1000.000000   1000.000000
mean    6300.617434          1.389105      0.019434
std     1697.095446         12.859576      0.002212
min     3873.477160          0.000000      0.003927
25%     5379.147649          0.000000      0.019122
50%     6100.950273          0.000000      0.019664
75%     6641.603255          0.000000      0.020045
max    24803.048244        199.828129      0.022887
Number of infeasible solution: 19
